# for_mnm_matching_algorithm



### 라이브러리 불러오기

In [31]:
import pandas as pd
import numpy as np

### 설문조사 데이터 불러오기

In [14]:
df = pd.read_excel("하우스 메이트 설문결과.xlsx")
df.head()

,TIMESTAMP,1. 성별,2. 나이,3. mbti,4. 흡연 유무,4-1. 원하는 메이트의 흡연여부,5. 반려동물과 같이 살 예정이다.,5-1. 반려동물을 키울 경우 어떤 동물인가?,5-2. 메이트의 반려동물 중 허용가능한 것은?,6. 난방/냉방 관련 문제,"7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)",7-1. 원하는 메이트,8.집에서 밥을 먹을 경우의 식사 스타일,8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등),9. 선호하는 식사유형,10. 메이트와 물건을 공유해도 괜찮습니까?,11. 방내에서의 소음문제,12. 음주 문제,13. 정리정돈을 신경쓰는 정도,14. 지인 출입
0,2021-10-20 11:29:52,여자,24 ~ 26,infp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,절대 못잡는다,NaN,벌레를 잡을 수 있어야만 한다,직접 요리,NaN,그렇다,NaN,NaN,소음은 없었으면 좋겠다.,2,누구든 데려오면 안된다.
1,2021-10-22 01:16:34,남자,27 ~ 29,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,내 공간만 아니면 신경쓰이지 않는다,미리 말만 하면 지인이 출입해도 상관없다.
2,2021-10-22 01:29:18,여자,20 ~ 23,esfp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지\t조류,더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\...,절대 못잡는다,벌레를 잡을 수 있어야만 한다,"외부 음식(포장, 배달)",NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 동영상/음악 소리는 신경쓰인다\t지속적인 알람소리는 신경쓰인다,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
3,2021-10-22 01:31:02,남자,20 ~ 23,모른다,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,NaN,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,NaN,같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
4,2021-10-22 01:33:35,남자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,과한 난방은 답답해하는 편이다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,괜찮지 않다(안된다),지속적인 알람소리는 신경쓰인다,메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보...,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   TIMESTAMP                              72 non-null     datetime64[ns]
 1   1. 성별                                  72 non-null     object        
 2   2. 나이                                  72 non-null     object        
 3   3. mbti                                72 non-null     object        
 4   4. 흡연 유무                               72 non-null     object        
 5   4-1. 원하는 메이트의 흡연여부                     72 non-null     object        
 6   5. 반려동물과 같이 살 예정이다.                    72 non-null     object        
 7   5-1. 반려동물을 키울 경우 어떤 동물인가?              15 non-null     object        
 8   5-2. 메이트의 반려동물 중 허용가능한 것은?             72 non-null     object        
 9   6. 난방/냉방 관련 문제                         71 non-null     object      

### 컬럼 이름 변경
- 컬럼 이름을 데이터베이스 테이블 명세서에 맞춰 변경한다.
- time 컬럼을 제외하면 총 19개의 컬럼을 가진다.

In [16]:
df.columns = ['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking', 'user_pet', 'user_pet_category', 'mate_pet_category', 
              'air', 'user_bug_killer', 'mate_bug_killer', 'user_cooking', 'mate_cooking', 'eat_together', 'share_item', 'noise', 'mate_alcohol', 
              'mate_clean', 'permission_to_enter']
df.head()

,time,sex,age,mbti,user_smoking,mate_smoking,user_pet,user_pet_category,mate_pet_category,air,user_bug_killer,mate_bug_killer,user_cooking,mate_cooking,eat_together,share_item,noise,mate_alcohol,mate_clean,permission_to_enter
0,2021-10-20 11:29:52,여자,24 ~ 26,infp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,절대 못잡는다,NaN,벌레를 잡을 수 있어야만 한다,직접 요리,NaN,그렇다,NaN,NaN,소음은 없었으면 좋겠다.,2,누구든 데려오면 안된다.
1,2021-10-22 01:16:34,남자,27 ~ 29,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,내 공간만 아니면 신경쓰이지 않는다,미리 말만 하면 지인이 출입해도 상관없다.
2,2021-10-22 01:29:18,여자,20 ~ 23,esfp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지\t조류,더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\...,절대 못잡는다,벌레를 잡을 수 있어야만 한다,"외부 음식(포장, 배달)",NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 동영상/음악 소리는 신경쓰인다\t지속적인 알람소리는 신경쓰인다,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
3,2021-10-22 01:31:02,남자,20 ~ 23,모른다,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,NaN,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,NaN,같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
4,2021-10-22 01:33:35,남자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,과한 난방은 답답해하는 편이다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,괜찮지 않다(안된다),지속적인 알람소리는 신경쓰인다,메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보...,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


### 컬럼 설명
- sex : 여자, 남자
- age : 10대, 20 ~ 23, 24 ~ 26, 27 ~ 29, 30대, 40대
- mbti : 따로 처리 필요
- user_smoking : 흡연자, 비흡연자
- mate_smoking : 흡연자, 비흡연자, 상관없음
  - 상관없음의 경우 해당 컬럼을 사용하지 않는 경우와 중간값을 주는 두가지의 방법을 사용할 수 있다.
- user_pet : 그렇다, 아니다
- user_pet_category : 강아지, 고양이, '파충류, 어류', 설치류, 조류, '기타'
  - 
- mate_pet_category : 어떠한 동물도 안된다., 어떤 동물이든 괜찮다., 고양이, 강아지, '파충류, 어류', 설치류, 조류
  - 어떠한 동물도 안된다. : user_pet == '그렇다' 일 경우를 제외한다.
  - 어떤 동물이든 괜찮다. : 
- air : 더위를 많이 타서 에어컨을 일찍부터 튼다, 여름에 항상 에어컨을 켜놓는다 = air_like_airconditionner / 에어컨을 틀고 못잔다 = air_night_airconditioner / 추위를 많이 타서 난방을 일찍부터 튼다, 과한 난방은 답답해하는 편이다 = air_like_heater / 해당하는 상황 없음 = air = none
- user_bug_killer : 잡는데 거리낌이 없다, 시키면 잡을 수 있다, 절대 못잡는다
- mate_bug_killer : 벌레를 잡을 수 있어야 한다, 벌레를 못잡아도 상관없다
- user_cooking : 직접 요리, 외부 음식(포장, 배달)
- mate_cooking : 불편하다, 신경쓰지 않는다(불편하지 않다)
- eat_together : 혼자 먹는 걸 선호한다, 같이 먹는 걸 선호한다
- shre_item : 말 안하고 사용해도 괜찮다, 사용하기 전 허락을 받으면 괜찮다, 괜찮지 않다(안된다)
- noise : 신경쓰지 않는다. = noise_none / 10분 이상의 말소리(통화/게임)은 신경쓰인다, 10분 이전의 말소리(통화/게임)도 신경쓰인다 = noise_talking / 10분 이상의 동영상/음악 소리는 신경쓰인다, 10분 이전의 동영상/음악 소리도 신경쓰인다 = noise_music / 지속적인 알람소리는 신경쓰인다 = noise_alarm
- mate_alcohol : 메이트의 술 문제는 신경쓰지 않는다, 메이트가 술 냄새를 풍기는 것은 싫다, 메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다, 메이트가 술주정만 하지 않으면 된다
- mate_clean : 내 공간만 아니면 신경쓰이지 않는다, 공유하는 공간의 경우 신경쓰인다, 메이트의 공간까지 신경쓰인다
- permission_to_enter : 본인이 없을 때 말없이 지인이 잠깐 출입해도 상관없다., 미리 말만 하면 지인이 출입해도 상관없다., 누구든 데려오면 안된다.

In [42]:
df.isnull().sum()

time                         0
sex                          0
age                          0
mbti                         0
user_smoking                 0
mate_smoking                 0
user_pet                     0
user_pet_category           57
mate_pet_category            0
air                          1
user_bug_killer              1
mate_bug_killer              0
user_cooking                 0
mate_cooking                 8
eat_together                 0
share_item                   1
noise                        2
mate_alcohol                 1
mate_clean                   0
permission_to_enter          0
air_like_airconditioner      0
air_night_airconditioner     0
air_like_heater             19
air_none                     0
dtype: int64

- air

In [20]:
df['air'].value_counts()

과한 난방은 답답해하는 편이다                                                                     12
더위를 많이 타서 에어컨을 일찍부터 튼다\t과한 난방은 답답해하는 편이다                                              8
해당하는 상황 없음                                                                            8
추위를 많이 타서 난방을 일찍부터 튼다                                                                 6
더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난방은 답답해하는 편이다                            6
더위를 많이 타서 에어컨을 일찍부터 튼다                                                                5
더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다                                         4
여름에 항상 에어컨을 켜놓는다\t과한 난방은 답답해하는 편이다                                                    4
더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\t과한 난방은 답답해하는 편이다                       3
여름에 항상 에어컨을 켜놓는다                                                                      3
추위를 많이 타서 난방을 일찍부터 튼다\t과한 난방은 답답해하는 편이다                                               3
에어컨을 틀고 못잔다\t과한 난방은 답답해하는 편이다   

In [44]:
# NaN -> '해당하는 상황 없음'
df['air'] = df['air'].fillna('해당하는 상황 없음')

In [28]:
df['air_like_airconditioner'] = df['air'].apply(lambda x : 1 if '더위를 많이 타서 에어컨을 일찍부터 튼다' in str(x) or '여름에 항상 에어컨을 켜놓는다' in str(x) else 0)
df['air_like_airconditioner'].value_counts()

0    37
1    35
Name: air_like_airconditioner, dtype: int64

In [25]:
df['air_night_airconditioner'] = df['air'].apply(lambda x : 1 if '에어컨을 틀고 못잔다' in str(x) else 0)
df['air_night_airconditioner'].value_counts()

0    66
1     6
Name: air_night_airconditioner, dtype: int64

In [32]:
def func(x):
  if '추위를 많이 타서 난방을 일찍부터 튼다' in str(x):
    return 1
  elif '과한 난방은 답답해하는 편이다'in str(x):
    return 0
  else:
    return np.nan

df['air_like_heater'] = df['air'].apply(lambda x : func(x))
df['air_like_heater'].value_counts()

0.0    33
1.0    20
Name: air_like_heater, dtype: int64

In [45]:
df['air_none'] = df['air'].apply(lambda x : 1 if '해당하는 상황 없음' in str(x) else 0)
df['air_none'].value_counts()

0    63
1     9
Name: air_none, dtype: int64

In [46]:
# air 삭제
df = df.drop(columns='air')
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'user_pet_category', 'mate_pet_category', 'user_bug_killer',
       'mate_bug_killer', 'user_cooking', 'mate_cooking', 'eat_together',
       'share_item', 'noise', 'mate_alcohol', 'mate_clean',
       'permission_to_enter', 'air_like_airconditioner',
       'air_night_airconditioner', 'air_like_heater', 'air_none'],
      dtype='object')

- noise
  - 신경쓰지 않는다. = noise_none / 10분 이상의 말소리(통화/게임)은 신경쓰인다, 10분 이전의 말소리(통화/게임)도 신경쓰인다 = noise_talking / 10분 이상의 동영상/음악 소리는 신경쓰인다, 10분 이전의 동영상/음악 소리도 신경쓰인다 = noise_music / 지속적인 알람소리는 신경쓰인다 = noise_alarm

In [47]:
# NaN -> '신경쓰지 않는다.'
df['noise'] = df['noise'].fillna('신경쓰지 않는다.')

In [64]:
def func(x):
  if '10분 이전의 말소리(통화/게임)도 신경쓰인다' in str(x):
    return 1
  elif '10분 이상의 말소리(통화/게임)은 신경쓰인다'in str(x):
    return 0.5
  elif '신경쓰지 않는다.' in str(x):
    return 0
  else:
    return np.nan

df['noise_talking'] = df['noise'].apply(lambda x : func(x))
df['noise_talking'].value_counts()

0.5    24
0.0    17
1.0     5
Name: noise_talking, dtype: int64

In [70]:
# 결측치 확인
df['noise_talking'].isnull().sum()

0

In [68]:
# 범주형이므로 결측치를 mode로 채움
df['noise_talking'] = df['noise_talking'].fillna(df['noise_talking'].mode().values[0])
df['noise_talking'].value_counts()

0.5    50
0.0    17
1.0     5
Name: noise_talking, dtype: int64

In [60]:
def func(x):
  if '10분 이전의 동영상/음악 소리도 신경쓰인다' in str(x):
    return 1
  elif '10분 이상의 동영상/음악 소리는 신경쓰인다'in str(x):
    return 0.5
  elif '신경쓰지 않는다.' in str(x):
    return 0
  else:
    return np.nan

df['noise_music'] = df['noise'].apply(lambda x : func(x))
df['noise_music'].value_counts()

0.5    24
0.0    17
1.0     5
Name: noise_music, dtype: int64

In [72]:
df['noise_music'].isnull().sum()

0

In [71]:
# 범주형이므로 결측치를 mode로 채움
df['noise_music'] = df['noise_music'].fillna(df['noise_music'].mode().values[0])
df['noise_music'].value_counts()

0.5    50
0.0    17
1.0     5
Name: noise_music, dtype: int64

In [56]:
df['noise_alarm'] = df['noise'].apply(lambda x : 1 if '지속적인 알람소리는 신경쓰인다' in str(x) else 0)
df['noise_alarm'].value_counts()

1    46
0    26
Name: noise_alarm, dtype: int64

In [73]:
df['noise_alarm'].isnull().sum()

0

In [74]:
# noise 삭제
df = df.drop(columns='noise')
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'user_pet_category', 'mate_pet_category', 'user_bug_killer',
       'mate_bug_killer', 'user_cooking', 'mate_cooking', 'eat_together',
       'share_item', 'mate_alcohol', 'mate_clean', 'permission_to_enter',
       'air_like_airconditioner', 'air_night_airconditioner',
       'air_like_heater', 'air_none', 'noise_talking', 'noise_music',
       'noise_alarm'],
      dtype='object')